## Introdução a Redes Neurais artificiais

In [7]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

### Implementação do perceptron

In [73]:
class Perceptron:
    def __init__(self, input_size, learning_rate = 0.1, n_epochs = 10):
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        
        # Inicialização dos pesos e do viés (bias)
        self.weights = np.zeros(input_size)
        self.bias = 0.0

    
    def activation(self, x):
        # Função degrau (Heaviside)
        return 1 if x >= 0 else 0


    def fit(self, X, y):
        for epoch in range(self.n_epochs):
            for xi, target in zip(X, y):
                prediction = self.predict(xi)
                error = target - prediction
                # Atualiza pesos e viés
                self.weights += self.learning_rate * error * xi
                self.bias += self.learning_rate * error

    
    def predict(self, x):
        linear_output = np.dot(x, self.weights) + self.bias
        
        return self.activation(linear_output)

In [74]:
# Exemplo com operação lógica AND
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([0, 0, 0, 1])

perceptron = Perceptron(input_size=2, learning_rate=0.1, n_epochs=10)
perceptron.fit(X, y)

for sample in X:
    print(f"Entrada: {sample} -> Predição: {perceptron.predict(sample)}")

Entrada: [0 0] -> Predição: 0
Entrada: [0 1] -> Predição: 0
Entrada: [1 0] -> Predição: 0
Entrada: [1 1] -> Predição: 1


###  Implementando o MLP com uma camada

In [80]:
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate = 0.5, epochs = 1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        
        # Inicialização dos pesos e biases
        np.random.seed(42)
        self.W1 = np.random.uniform(-1, 1, (input_size, hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        
        self.W2 = np.random.uniform(-1, 1, (hidden_size, output_size))
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def fit(self, X, y):
        for epoch in range(self.epochs):
            # Forward
            # Camada oculta
            z1 = np.dot(X, self.W1) + self.b1
            a1 = self.sigmoid(z1)
            
            # Camada de saída
            z2 = np.dot(a1, self.W2) + self.b2
            a2 = self.sigmoid(z2)  # saída final
            
            # Calcular erro
            error = y - a2
            
            # Backpropagation
            delta2 = error * self.sigmoid_derivative(a2)
            delta1 = delta2.dot(self.W2.T) * self.sigmoid_derivative(a1)
            
            # Atualização dos pesos e bias
            self.W2 += a1.T.dot(delta2) * self.learning_rate
            self.b2 += np.sum(delta2, axis = 0, keepdims = True) * self.learning_rate
            
            self.W1 += X.T.dot(delta1) * self.learning_rate
            self.b1 += np.sum(delta1, axis = 0, keepdims = True) * self.learning_rate
            
            # Exibir perda a cada 1000 épocas
            if epoch % 1000 == 0:
                loss = np.mean(np.abs(error))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, X):
        z1 = np.dot(X, self.W1) + self.b1
        a1 = self.sigmoid(z1)
        
        z2 = np.dot(a1, self.W2) + self.b2
        a2 = self.sigmoid(z2)
        return a2

In [81]:
# Exemplo com a lógica [XOR]
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])
y = np.array([[0], [1], [1], [0]])


mlp = MLP(input_size = 2, hidden_size = 2, output_size = 1, learning_rate = 0.5, epochs = 10000)
mlp.fit(X, y)

Epoch 0, Loss: 0.5005
Epoch 1000, Loss: 0.4922
Epoch 2000, Loss: 0.0987
Epoch 3000, Loss: 0.0591
Epoch 4000, Loss: 0.0454
Epoch 5000, Loss: 0.0381
Epoch 6000, Loss: 0.0333
Epoch 7000, Loss: 0.0300
Epoch 8000, Loss: 0.0275
Epoch 9000, Loss: 0.0255


In [82]:
output = mlp.predict(X)

print("\nSaída prevista após treinamento:")
print(np.round(output, 3))


Saída prevista após treinamento:
[[0.023]
 [0.974]
 [0.974]
 [0.021]]


### Exemplo com rede MLP 

#### Abertura da base de dados

In [10]:
bc = datasets.load_breast_cancer()
x = bc.data
y = bc.target

#### Divisão do conjunto de dados entre treino e teste

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, shuffle = True, random_state = 1, stratify = y)

In [12]:
print('Labels counts in y:', np.bincount(y))
print('Labels counts in y_train:', np.bincount(y_train))
print('Labels counts in y_test:', np.bincount(y_test))

Labels counts in y: [212 357]
Labels counts in y_train: [148 250]
Labels counts in y_test: [ 64 107]


#### Normalização dos dados

In [21]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)

scaler.fit(X_test)
X_test_std = scaler.transform(X_test)

#### Treinamento de uma rede MLP da biblioteca sklearn

In [36]:
# Rede neural com uma camada oculta de 30 neurônios
model = MLPClassifier(hidden_layer_sizes = (30, ), activation = 'logistic', 
                      solver = 'adam', max_iter = 1000, random_state = 42)

model.fit(X_train_std, y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=(30,), max_iter=1000,
              random_state=42)

#### Predição e avaliação

In [37]:
y_pred = model.predict(X_test_std)

In [38]:
accuracy_score(y_test, y_pred)

0.9707602339181286